In [192]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import nltk
nltk.download('stopwords')
from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils
from sklearn.model_selection import train_test_split


from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

questions = pd.read_csv("data/train_dataset.csv", header=None, encoding="iso-8859-1", sep=";")
questions.columns = ['id', 'question', 'answer', 'topic']



REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

questions['question'] = questions['question'].apply(clean_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/silvan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [193]:
X = questions.question
y = questions.topic
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

train_posts = X_train
train_tags = y_train

test_posts = X_test
test_tags = y_test

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)

tokenize.fit_on_texts(train_posts) # only fit on train
x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)


encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

batch_size = 32
epochs = 2


# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)


val_score = model.evaluate(x_train, y_train,
                       batch_size=batch_size, verbose=1)
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Train acc:',val_score[1] )
print('Test accuracy:', score[1])

Train on 6417 samples, validate on 713 samples
Epoch 1/2
6417/6417 [==============================] - 8s 1ms/step - loss: 0.8006 - acc: 0.7201 - val_loss: 0.4133 - val_acc: 0.8710
Epoch 2/2
1783/1783 [==============================] - 0s 80us/step
Train acc: 0.9357643758932972
Test accuracy: 0.8844643860002348


In [195]:
generated_questions = pd.read_csv("data/crowdanswers.tsv", encoding="ISO-8859-1", delimiter="\t", na_filter=False)
generated_questions.columns = ['id', 'question', 'answer', 'difficulty', 'opinion', 'factuality']

labels = sorted(['science-technology', 'for-kids', 'video-games', 'sports', 'music'])

generated_questions['question'].apply(clean_text)
x_predict = tokenize.texts_to_matrix(generated_questions['question'])
# result = model.predict(x_predict)
result= model.predict_classes(x_predict, batch_size=1)
#y_classes = result.argmax(axis=-1)
predictated_labels = [labels[i] for i in result]
# predicted_label = sorted(labels)[result]
output = pd.DataFrame(data={"id":generated_questions["id"], "question":generated_questions["question"], "answer":generated_questions["answer"], "difficulty":generated_questions["difficulty"], "opinion":generated_questions["opinion"], "factuality":generated_questions["factuality"], "topic":predictated_labels})
output.to_csv('generated.csv',encoding='utf-8', index=False)

In [200]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
from gensim.models import KeyedVectors
from keras.backend.tensorflow_backend import set_session
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.layers import Dense, Embedding, Dropout, Flatten, Activation
from tensorflow.python.keras.layers import LSTM, SimpleRNN
from tensorflow.python.keras import optimizers
from tensorflow.python.keras.preprocessing import text as keras_text, sequence as keras_seq
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score
from sklearn.utils import shuffle
from tensorflow import set_random_seed
import gc
import os
from keras.preprocessing.sequence import pad_sequences

EMBEDDING_SIZE=300
WORDS_SIZE=8000
INPUT_SIZE=100
NUM_CLASSES=5
EPOCHS=10

questions = pd.read_csv("data/train_dataset.csv", header=None, encoding="iso-8859-1", sep=";")
questions.columns = ['id', 'question', 'answer', 'topic']

def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

questions['question'] = questions['question'].apply(clean_text)

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)


tokenizer = keras_text.Tokenizer(char_level=False)
tokenizer.fit_on_texts(x_train)
tokenizer.num_words=WORDS_SIZE

maxlen = 100
## Tokkenizing train data and create matrix
list_tokenized_train = tokenizer.texts_to_sequences(x_train)
x_train = pad_sequences(list_tokenized_train, padding='post', maxlen=maxlen)

## Tokkenizing test data and create matrix
list_tokenized_test = tokenizer.texts_to_sequences(x_test)
x_test = pad_sequences(list_tokenized_test, padding='post', maxlen=maxlen)

encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

y_train = utils.to_categorical(y_train, NUM_CLASSES)
y_test = utils.to_categorical(y_test, NUM_CLASSES)

In [197]:
word2vec = KeyedVectors.load_word2vec_format('word2vec/GoogleNews-vectors-negative300.bin.gz', binary=True)
word_index = tokenizer.word_index

vocabulary_size=min(len(word_index)+1,8000)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_SIZE))
for word, i in word_index.items():
    if i>=WORDS_SIZE:
        continue
    try:
        embedding_vector = word2vec[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_SIZE)

embedding_layer = Embedding(vocabulary_size,
                            EMBEDDING_SIZE,
                            weights=[embedding_matrix],
                            input_length=INPUT_SIZE,
                            trainable=False)

In [201]:
model = Sequential(name='Word2Vec LSTM')

model.add(embedding_layer)
model.add(LSTM(EMBEDDING_SIZE))
model.add(Dropout(0.5))
model.add(Dense(5))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [ ]:
history = model.fit(x = x_train,
          y = y_train,
          validation_data = (x_test, y_test),
          epochs = EPOCHS,
          batch_size = 128,
          verbose =1)


val_score = model.evaluate(x_train, y_train, batch_size=batch_size, verbose=1)
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)

Train on 7130 samples, validate on 1783 samples
Epoch 1/10
7130/7130 [==============================] - 92s 13ms/sample - loss: 1.4573 - categorical_accuracy: 0.3544 - val_loss: 1.3593 - val_categorical_accuracy: 0.4173
Epoch 2/10
7130/7130 [==============================] - 90s 13ms/sample - loss: 1.3861 - categorical_accuracy: 0.4107 - val_loss: 1.3569 - val_categorical_accuracy: 0.4173
Epoch 3/10
7130/7130 [==============================] - 99s 14ms/sample - loss: 1.3838 - categorical_accuracy: 0.4116 - val_loss: 1.3576 - val_categorical_accuracy: 0.4173
Epoch 4/10
4608/7130 [==================>...........] - ETA: 35s - loss: 1.3760 - categorical_accuracy: 0.4160

In [189]:
generated_questions = pd.read_csv("data/crowdanswers.tsv", encoding="ISO-8859-1", delimiter="\t", na_filter=False)
generated_questions.columns = ['id', 'question', 'answer', 'difficulty', 'opinion', 'factuality']

labels = sorted(['science-technology', 'for-kids', 'video-games', 'sports', 'music'])

list_tokenized = tokenizer.texts_to_sequences(generated_questions['question'])
x_predict = pad_sequences(list_tokenized, padding='post', maxlen=maxlen)

result= model.predict_classes(x_predict, batch_size=128)
predictated_labels = [labels[i] for i in result]
print(predictated_labels)
# predicted_label = sorted(labels)[result]
output = pd.DataFrame(data={"id":generated_questions["id"], "question":generated_questions["question"], "answer":generated_questions["answer"], "difficulty":generated_questions["difficulty"], "opinion":generated_questions["opinion"], "factuality":generated_questions["factuality"], "topic":predictated_labels})
output.to_csv('generated.csv',encoding='utf-8', index=False)

['music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 'music', 